# Use case: ERA5 small area 

Earth Data Hub offers an innovative and super-efficient way to access data.

Here we present how to best access the service in the simplest use case.

## Setup the environment

**If you haven't done it already, follow the [Getting started notebook](./00-getting-started.ipynb) to setup your environment and DestinE credentials.**

## Download 30 years of an ERA5 variable on a small area

Our use case is to compute various climatological averages of one ERA5 variable over a long period of time, e.g. the surface temperature over 30 years. Note that we will be  using the ERA5 single levels dataset that has more than 100 variable and every variable is almost 3TB in size. 

The best practice for downloading data form the Earth Data Hub comprise the following steps:
1. open the dataset using the code snippet found on the [ERA5 dataset page](https://earthdatahub.destine.eu/collections/era5/datasets/reanalysis-era5-single-levels)
2. select the variable
3. select the area of interest (optionally alligning it on chunk boundaries)
4. select the time interval of interest
5. download the data to memory (with `.persist()` or `.compute()`)
6. save the data or compute the result in memory

### Open the dataset

The following assumes you set up the EDH Personal Access Token in your _netrc_ file.

In [ ]:
import xarray as xr

era5_single_levels_dataset = xr.open_dataset(
    "https://data.earthdatahub.destine.eu/era5/reanalysis-era5-single-levels-v0.zarr",
    storage_options={"client_kwargs": {"trust_env": True}},
    chunks={},
    engine="zarr",
).drop_vars(["surface", "number"])
era5_single_levels_dataset

### Select and prepare the variable

Note that all operation are lazy, that is are not applied to the whole 3TB of data, but are just recorded for later use. Download and computations are only done when requested.

In [ ]:
t2m_world = era5_single_levels_dataset.t2m
t2m_world

### Select the area of interest

For example we are interested in the area of the Alps.

This a very convenient example as:
* it is a large enough area to be more interesting than a time-series of a single point,
* it is small enough to fit a single Zarr chunk, so the notebook can be run even with a slow internet connection,
* when plotted it is easy to identify even without adding coastlines and country borders that will make the notebook more complex.

In [ ]:
aoi_selection = {
    "latitude": slice(49, 43),  # NOTE: ERA5 has a descending latitude coordinate
    "longitude": slice(5, 15),
}
t2m_aoi = t2m_world.sel(aoi_selection)
t2m_aoi

Note that the data size is not much smaller, we went from 3TB for global data to 3GB of our small area of interest.

We plot the map of the temperature at one time to double check that the selection is correct:

In [ ]:
t2m_aoi.sel(valid_time="2020-01-01T00:00:00").plot()

### Optional: allign the area of interest on chunk boundaries

This a pro move and you can skip it.

When accessing the data in Zarr you always dowload whole chunks, even if you are only interestind in part of them. In the case of ERA5 single level the spatial chunks are `(64, 64)` in size and even if you can read above that your DataArray in only 2GB of data you are most probably donwloading more data and then throwing a part of it away.

You can use the following (not very nice) code to grow your area of interest to the boundaries of the Zarr chunks, so you use all the data you donwload. Note that now you also have the size of the data that would be downloaded is it was not compressed (the data is in fact compressed so actual downlaod is smaller).

In [ ]:
def align_indexer(step, indexer):
    if not isinstance(indexer, slice):
        return indexer
    assert indexer.step is None
    start = indexer.start // step * step if indexer.start else indexer.start
    stop = (indexer.stop // step + 1) * step if indexer.stop else indexer.stop
    return slice(start, stop)


query_results = xr.core.indexing.map_index_queries(
    t2m_world, indexers=aoi_selection, method=None, tolerance=None
)
print(query_results.dim_indexers)

aoi_iselection = {
    dim: align_indexer(64, query_results.dim_indexers[dim])
    for dim in query_results.dim_indexers
}
print(aoi_iselection)

t2m_aoi = t2m_world.isel(aoi_iselection)
t2m_aoi

So, the full time-series for a whole chunk is really 11.5GB, not 2GB.

Let's have a look at the area of the whole chunk.

In [ ]:
t2m_aoi.sel(valid_time="2020-01-01T00:00:00").plot()

Great! A single 64x64 chunk gets a big part of central Europe.

### Select the time interval of interest

We take one of the typical 30 year periods to compute climatologies and finally we have a definition of the data we want to download.

In [ ]:
t2m_aoi_toi = t2m_aoi.sel(valid_time=slice("1981", "2010"))
t2m_aoi_toi

From the representation above we learn a few things:
1. the uncompressed data to be downloaded, e.g. 4GB (compression depends on the dataset and the variable)
2. the number of chunks to be downloaded, e.g. 62

### Download the data to memory

Finally we are ready to download only the data that we are interested in, in memory.

The best practice is to call the `.compute()` method load the data into a numpy array in memory.

**This operation is the slow one. It takes up to 20 minutes on a 8 Mbps connection.**

It depends on the download speed of your internet connection and on the load on the Earth Data Hub. The closer you are to the data the faster it is, and this is one of the reason the EDH is best suited to be used from within the DestinE platform.

In [ ]:
%%time

t2m_aoi_toi_data = t2m_aoi_toi.compute()

### Perform any computation

#### Prepare the raw data 

In [ ]:
t2m_aoi_toi_data_C = t2m_aoi_toi_data - 273.15
t2m_aoi_toi_data_C.attrs["units"] = "°C"
t2m_aoi_toi_data_C.attrs["long_name"] = "surface temperature"
t2m_aoi_toi_data_C

#### Monthly climatology

In [ ]:
%%time

t2m_aoi_toi_month_mean = t2m_aoi_toi_data_C.groupby("valid_time.month").mean()
t2m_aoi_toi_month_mean

In [ ]:
t2m_aoi_toi_month_mean.plot(vmin=-30, col="month", col_wrap=3)

#### Daily climatological mean and quintile extreemes

In [ ]:
%%time

t2m_aoi_toi_doy_mean = t2m_aoi_toi_data_C.groupby("valid_time.dayofyear").mean()
t2m_aoi_toi_doy_mean = t2m_aoi_toi_doy_mean.sel(dayofyear=slice(0, 365))
t2m_aoi_toi_doy_mean

In [ ]:
point = {
    "latitude": 46.5,
    "longitude": 11.8,
    "method": "nearest",
}

t2m_point_doy_mean = t2m_aoi_toi_doy_mean.sel(**point)

t2m_point_doy_mean.plot()

In [ ]:
%%time

t2m_aoi_toi_doy_quantile = t2m_aoi_toi_data_C.groupby("valid_time.dayofyear").quantile(
    [0.9, 0.5, 0.1]
)
t2m_aoi_toi_doy_quantile = t2m_aoi_toi_doy_quantile.sel(dayofyear=slice(0, 365))
t2m_aoi_toi_doy_quantile

In [ ]:
from matplotlib import pyplot as plt

_, ax = plt.subplots()

t2m_point_doy_quantile = t2m_aoi_toi_doy_quantile.sel(**point)

ax.fill_between(
    t2m_point_doy_quantile.dayofyear,
    t2m_point_doy_quantile.sel(quantile=0.9),
    t2m_point_doy_quantile.sel(quantile=0.1),
    alpha=0.3,
)

t2m_point_doy_quantile.sel(quantile=0.5).plot(ax=ax, alpha=0.5)
t2m_point_doy_mean.plot(ax=ax)